In [134]:
import requests as req
url = "https://api.bithumb.com/v1/candles/minutes/1?market=KRW-BTC&count=1"
d_url = "https://api.bithumb.com/v1/candles/days/1?market=KRW-BTC&count=1"
re = req.get(url).json()[0]
d_re = req.get(d_url).json()[0]

max_p = float(re['high_price'])
min_p = float(re['low_price'])
st_p = float(re['opening_price'])


print()


[{'market': 'KRW-BTC', 'candle_date_time_utc': '2026-02-06T01:25:00', 'candle_date_time_kst': '2026-02-06T10:25:00', 'opening_price': 95377000, 'high_price': 95379000, 'low_price': 95377000, 'trade_price': 95377000, 'timestamp': 1770341105000, 'candle_acc_trade_price': 10305253.71069, 'candle_acc_trade_volume': 0.10804757, 'unit': 1}]


103,732,051원


In [153]:
Me = 'java'
comA = ['Java' , 'HTML' , 'javascript','ORACLE']
comB = ['Html' , 'Python' , 'C','Mysql']
comC = ['Java' , 'CSS' , 'javascript', 'C++','ORACLE' , 'PYTHON']




# print(upper(comA[0]))
for i in comA:
    comA = []
    comA.append(i)


# print("합격" if Me in comA else "불합격", '니똥굵다')
# print("합격" if Me in comB else "불합격", '니똥굵다')
# print("합격" if Me in comC else "불합격", '니똥굵다')

java


In [11]:
import requests as req
import webbrowser as wb

# 위성위치
url = "http://api.open-notify.org/iss-now.json"
# 탑승자 명단
url2 = "http://api.open-notify.org/astros.json"

result = req.get(url).json()

print(result)
lat = result['iss_position']['latitude'] # 위도
lon = result['iss_position']['longitude'] # 경도

human_num = req.get(url2).json()["number"]

url = f"https://www.google.com/maps?q={lat},{lon}"
print(f"현재 {human_num}명이 탑승한 인공위성(국제우주정거장) 위치를 열겠습니다.위도:{lat}, 경도:{lon}")
wb.open(url)

{'timestamp': 1770354497, 'iss_position': {'latitude': '8.8014', 'longitude': '21.5709'}, 'message': 'success'}
현재 12명이 탑승한 인공위성(국제우주정거장) 위치를 열겠습니다.위도:8.8014, 경도:21.5709


True

gio: https://www.google.com/maps?q=8.8014,21.5709: Operation not supported


In [13]:
import requests as req

url = "https://api.open-meteo.com/v1/forecast?latitude=37.5665&longitude=126.9780&current_weather=true"

rs = req.get(url).json()
print(rs)

{'latitude': 37.55, 'longitude': 127.0, 'generationtime_ms': 0.06532669067382812, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 34.0, 'current_weather_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature': '°C', 'windspeed': 'km/h', 'winddirection': '°', 'is_day': '', 'weathercode': 'wmo code'}, 'current_weather': {'time': '2026-02-06T05:15', 'interval': 900, 'temperature': -4.9, 'windspeed': 9.6, 'winddirection': 304, 'is_day': 1, 'weathercode': 0}}


In [15]:
import re
import requests 
from bs4 import BeautifulSoup

URL = "https://www.pknu.ac.kr/main/399?action=view&no=723270"

def fetch_text(url: str) -> str:
    # 사이트가 가끔 UA 없으면 막는 경우가 있어 넣어줌
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                      "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    r = requests.get(url, headers=headers, timeout=10)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")
    # 본문 전체를 텍스트로 (줄바꿈 유지)
    return soup.get_text("\n")

def extract_section(text: str, start_title: str, end_title: str) -> str:
    start = text.find(start_title)
    if start == -1:
        raise ValueError(f"start_title not found: {start_title}")
    end = text.find(end_title, start)
    if end == -1:
        # 끝 타이틀 못 찾으면 페이지 끝까지
        end = len(text)
    return text[start:end]

def parse_daily_menu(section_text: str) -> dict:
    # 예: "2월 2일" 패턴
    date_pat = re.compile(r"(\d+월\s*\d+일)")
    lines = [ln.strip() for ln in section_text.splitlines() if ln.strip()]

    # 날짜가 나오는 라인 인덱스 수집
    date_idxs = []
    for i, ln in enumerate(lines):
        if date_pat.fullmatch(ln):
            date_idxs.append(i)

    if not date_idxs:
        raise ValueError("No date lines like '2월 2일' found in section")

    result = {}
    for idx_pos, i in enumerate(date_idxs):
        date = lines[i]
        j = date_idxs[idx_pos + 1] if idx_pos + 1 < len(date_idxs) else len(lines)

        # 날짜 다음 줄부터 다음 날짜 전까지 = 그날 메뉴 텍스트
        block = lines[i + 1 : j]

        # 너무 일반적인 안내/전화번호 같은 건 필요하면 필터링
        # (원하면 필터 규칙 더 촘촘히 잡아줄게)
        result[date] = block

    return result

if __name__ == "__main__":
    text = fetch_text(URL)

    # "미래관 라일락 레스토랑" 구간만 자르기
    section = extract_section(
        text,
        start_title="미래관 라일락 레스토랑",
        end_title="다래락 학생식당"
    )

    menu_by_day = parse_daily_menu(section)

    # 예: 2월 2일 메뉴만 출력
    day = "2월 2일"
    print(day)
    for item in menu_by_day.get(day, []):
        print("-", item)

ValueError: start_title not found: 미래관 라일락 레스토랑

In [79]:
import requests as rq
url = "https://api.fda.gov/drug/event.json?count=patient.reaction.reactionmeddrapt.exact"

rs = rq.get(url).json()



# aa = input("찾고싶은 질병의 이름을 입력하시오")
# bb = input("찾고싶은 질병의 이름을 입력하시오")
# cc = []

for item in rs['results']:
    if item.get("term") == aa :
        print('보고건의 수는 ' , item.get('count'),'개 입니다.')
        cc.append(item.get('count'))

for item in rs['results']:
    if item.get("term") == bb :
        print('보고건의 수는 ' , item.get('count'),'개 입니다.')
        cc.append(item.get('count'))

if int(cc[0]) - int(cc[1]) > 0:
   c_n = cc[0]
else:
   c_n = cc[1]
    

if cc[0] - cc[1] < 0:
    c = cc[1] - cc[0]
else:
    c = cc[0] - cc[1]
    
print(f"더 많이 발생된 질병은 {c_n} 이고 차이는 {c}건 입니다. ")



pda = rs['meta']['disclaimer'].count("PDA")


print(f"경고문구 안에 PDA의 개수는 {pda}개 입니다.")




보고건의 수는  171703 개 입니다.
보고건의 수는  171703 개 입니다.
더 많이 발생된 질병은 171703 이고 차이는 0건 입니다. 
경고문구 안에 PDA의 개수는 0개 입니다.


In [3]:
def is_odd(number):
    if number % 2 == 1:
        return True
    else :
        return False

is_odd(3)

True

In [9]:
def avg_numbers(*args):
    result = 0
    for i in args:
        result += i
    return result / len(args)

print(avg_numbers(1,2))


avg_numbers(1,2,3,4,5)


1.5


3.0

In [11]:
input1 = int(input('첫번째 숫자를 입력하세요'))
input2 = int(input('2번째 숫자를 입력하세요'))

total = input1 + input2
print('두수의 합은 %s입니다.' %total)

첫번째 숫자를 입력하세요 3
2번째 숫자를 입력하세요 6


두수의 합은 9입니다.


In [17]:
f1 = open("test.txt", 'w')
f1.write("Life is too short")
f1.close

f2 = open("test.txt" , 'r')
print(f2.read())

Life is too short


In [25]:
user_input = input("저장할 내용을 입력하세요:")
f = open('test.txt' , 'a')
f.write(user_input)
f.write('\n')
f.close()

저장할 내용을 입력하세요: 김재형


In [28]:
f = open('test.txt','r')
body = f.readline()
f.close()
body = body.replace('java' , 'python')
f = open('test.txt','w')
f.write(body)
f.close()

In [36]:
# C:\> cd doit
# C:\doit> python myargv.py 1 2 3 4 5 6 7 8 9 10

import sys
a = 0
args = sys.argv[1:]
for i in args:
    a += int(i)

print(a)

-f
/home/jhkim/.local/share/jupyter/runtime/kernel-c722e40a-ba76-481e-b554-f6750487223d.json


In [ ]:
input('뭘 쓸래? 쒝더 뻑뻑아')
with open('./test_note.txt'.'a') as f:
    f.write()
    

In [5]:
class Calculator:
    def __init__(self):
        
        self.result = 0
        print("aslk")

        
    def add(self , num):
        self.result += num
        return self.result



cal1 = Calculator()
cal2 = Calculator()


print(cal1.add(3))
print(cal1.add(4))
print(cal2.add(3))
print(cal2.add(5))
print(cal1.add(7))
print(cal1.add(8))

    

aslk
aslk
3
7
3
8
14
22


In [12]:
def add(x):
    print(x)
     

a = add(3)
print(a)

3
None


In [13]:
class DDuk:
    def __init__ (self,won,ea):
        self.ea = ea
        self.won = won

    def dduk(self):
        result = self.won * self.ea
        print(f"떡가격 수발거 {result}")

    def odeng(self):
        result = self.won * 2 * self.ea
        print(f"오뎅 가격 {result}")

In [25]:
class Pet:
    def __init__ (self,name, species):

        self.name = name
        self.species = species
        self.hunger = 50
        
        
    def feed(self , bab):
        if self.hunger > 10:
            self.hunger -=bab
        return f"{self.name}에게 밥을 줬어요. 배고픕니다"

    def play(self):
        if self.hunger < 70:
            self.hunger += 20
        return f"{self.name}에게 잘 놀았습니다.  배고픔 {self.hunger}"


dog1 = Pet('뽀삐','말티즈')
dog2 = Pet('불독','마리공주')
cat1 = Pet('고','양')
cat2 = Pet('냐','옹')



In [26]:
cat2.feed(10)

'냐에게 밥을 줬어요. 배고픕니다'

In [32]:
class Bank:
    interest_rate = 0.2
    def __init__(self, owner, bal):
        self.onwer = owner
        self.bal = bal

    def bprint(self):
        return f"{self.owner}은 {self.bal}를 가지고 있습니다. "

    def deposite(self, money):
        self.bal += money
        return f"{self.owner}은 {money}를 입금하였습니다 , 잔액은 {self.bal}얼마 입니다."

    def withdraw(self, money):
        if self.bal > money :
            self.bal -= money
            return f"{self.owner}은 {money}를 출금하였습니다. 잔액은 {self.bal}입니다."
        else: 
            print(" 잔액 이상을 출금하실 수 없습니다.")
        

    def interest(self):
        self.bal = self.bal * (1 + self.interest_rate)
        return f"이자를 받으셔서 잔액이{self.bal}가 되었습니다."

    
        

In [33]:
acc1 = Bank('재형',100000)
acc2 = Bank('준호',200000)
acc3 = Bank('서영',300000)

In [36]:
acc1.deposite()

TypeError: Bank.deposite() missing 1 required positional argument: 'money'

In [37]:
import random
class Character:
    def __init__(self, name, hp, attack_p):
        self.name = name
        self.hp = hp
        self.attack_p = attack_p
    def attack(self, opponent):
        demage = random.randint(2, self.attack_p)
        opponent -= demage
        print(f"{self.name} 가 {opponent.name}을 공격하여 {damage}를 입혔습니다.")
    def now_status(self):
        return f"이름 :{self.name} 체력:{self.hp}"
    def is_alive(self):
        return self.hp > 0
    
        

In [38]:
player = Character( " 영웅 ",  attack_p = 15 , hp = 50)
monter = Character( " 골램 ", hp = 60 , attack_p = 9)

In [ ]:
print(f"{전투시작 :=^30}")
while player.is_alive() and monster.is_alive():
    input("경기를 시작하면 엔터를 치세요")
    